In [35]:
from pathlib import Path
import numpy as np, pandas as pd, xarray as xr

In [36]:
base = Path("/media/julienb/Elements/Carmen/LMAN_project/LMANX_correlations_project/LMANX_behavior_data/BirdData/RedRed/Sessions/Rec_13_03_2022_g0-55200to56200/song/")
txts = list(base.glob("*.txt"))
txts

[PosixPath('/media/julienb/Elements/Carmen/LMAN_project/LMANX_correlations_project/LMANX_behavior_data/BirdData/RedRed/Sessions/Rec_13_03_2022_g0-55200to56200/song/redreclab__raw_chunk_12_annot_subtones.txt'),
 PosixPath('/media/julienb/Elements/Carmen/LMAN_project/LMANX_correlations_project/LMANX_behavior_data/BirdData/RedRed/Sessions/Rec_13_03_2022_g0-55200to56200/song/redreclab__raw_chunk_13_annot_subtones.txt'),
 PosixPath('/media/julienb/Elements/Carmen/LMAN_project/LMANX_correlations_project/LMANX_behavior_data/BirdData/RedRed/Sessions/Rec_13_03_2022_g0-55200to56200/song/redreclab__raw_chunk_14_annot_subtones.txt'),
 PosixPath('/media/julienb/Elements/Carmen/LMAN_project/LMANX_correlations_project/LMANX_behavior_data/BirdData/RedRed/Sessions/Rec_13_03_2022_g0-55200to56200/song/redreclab__raw_chunk_15_annot_subtones.txt'),
 PosixPath('/media/julienb/Elements/Carmen/LMAN_project/LMANX_correlations_project/LMANX_behavior_data/BirdData/RedRed/Sessions/Rec_13_03_2022_g0-55200to56200/s

In [37]:
annotations = [pd.read_csv(txt, header=None, names=["start", "end", "syb"]).assign(filename= str(txt.relative_to(base))) for txt in txts]
annotations = pd.concat(annotations)
annotations["start"] = annotations["start"].astype(int)
annotations["end"] = annotations["end"].astype(int)
# [df.insert(0, "filename", str(txt)) for df, txt in zip(annotations, txts)]
annotations

,start,end,syb,filename
0,5727,8702,b1,redreclab__raw_chunk_12_annot_subtones.txt
1,8703,11677,b2,redreclab__raw_chunk_12_annot_subtones.txt
2,11678,14654,b3,redreclab__raw_chunk_12_annot_subtones.txt
3,16672,26711,a,redreclab__raw_chunk_12_annot_subtones.txt
4,27661,30595,b1,redreclab__raw_chunk_12_annot_subtones.txt
...,...,...,...,...
14,276855,277780,NaN,redreclab__raw_chunk_9_annot_subtones.txt
15,365576,367078,NaN,redreclab__raw_chunk_9_annot_subtones.txt
16,429307,430150,NaN,redreclab__raw_chunk_9_annot_subtones.txt
17,467545,468315,NaN,redreclab__raw_chunk_9_annot_subtones.txt


In [38]:
annotations["chunk_num"] = annotations["filename"].str.extract(r'raw_chunk_(\d+)').astype(int)
annotations["npy_file"] = annotations["filename"].str.slice(0, -(len("_annot_subtones.txt"))) + ".npy"
annotations

,start,end,syb,filename,chunk_num,npy_file
0,5727,8702,b1,redreclab__raw_chunk_12_annot_subtones.txt,12,redreclab__raw_chunk_12.npy
1,8703,11677,b2,redreclab__raw_chunk_12_annot_subtones.txt,12,redreclab__raw_chunk_12.npy
2,11678,14654,b3,redreclab__raw_chunk_12_annot_subtones.txt,12,redreclab__raw_chunk_12.npy
3,16672,26711,a,redreclab__raw_chunk_12_annot_subtones.txt,12,redreclab__raw_chunk_12.npy
4,27661,30595,b1,redreclab__raw_chunk_12_annot_subtones.txt,12,redreclab__raw_chunk_12.npy
...,...,...,...,...,...,...
14,276855,277780,NaN,redreclab__raw_chunk_9_annot_subtones.txt,9,redreclab__raw_chunk_9.npy
15,365576,367078,NaN,redreclab__raw_chunk_9_annot_subtones.txt,9,redreclab__raw_chunk_9.npy
16,429307,430150,NaN,redreclab__raw_chunk_9_annot_subtones.txt,9,redreclab__raw_chunk_9.npy
17,467545,468315,NaN,redreclab__raw_chunk_9_annot_subtones.txt,9,redreclab__raw_chunk_9.npy


In [39]:
chunks = annotations[["chunk_num", "npy_file"]].drop_duplicates().reset_index(drop=True)
chunks = chunks.sort_values(by="chunk_num")
chunks["size"] = chunks["npy_file"].apply(lambda x: np.load(base / x).size)
chunks["cumulative_end"] = chunks["size"].cumsum()
chunks["cumulative_start"] = chunks["cumulative_end"].shift(1, fill_value=0)
chunks

,chunk_num,npy_file,size,cumulative_end,cumulative_start
56,0,redreclab__raw_chunk_0.npy,490845,490845,0
8,1,redreclab__raw_chunk_1.npy,490846,981691,490845
19,2,redreclab__raw_chunk_2.npy,490845,1472536,981691
30,3,redreclab__raw_chunk_3.npy,490846,1963382,1472536
41,4,redreclab__raw_chunk_4.npy,490845,2454227,1963382
...,...,...,...,...,...
55,62,redreclab__raw_chunk_62.npy,490846,30923270,30432424
59,63,redreclab__raw_chunk_63.npy,490845,31414115,30923270
60,64,redreclab__raw_chunk_64.npy,490846,31904961,31414115
61,65,redreclab__raw_chunk_65.npy,490845,32395806,31904961


In [40]:
all = pd.merge(annotations, chunks, on=["chunk_num", "npy_file"])
all

,start,end,syb,filename,chunk_num,npy_file,size,cumulative_end,cumulative_start
0,5727,8702,b1,redreclab__raw_chunk_12_annot_subtones.txt,12,redreclab__raw_chunk_12.npy,490846,6380992,5890146
1,8703,11677,b2,redreclab__raw_chunk_12_annot_subtones.txt,12,redreclab__raw_chunk_12.npy,490846,6380992,5890146
2,11678,14654,b3,redreclab__raw_chunk_12_annot_subtones.txt,12,redreclab__raw_chunk_12.npy,490846,6380992,5890146
3,16672,26711,a,redreclab__raw_chunk_12_annot_subtones.txt,12,redreclab__raw_chunk_12.npy,490846,6380992,5890146
4,27661,30595,b1,redreclab__raw_chunk_12_annot_subtones.txt,12,redreclab__raw_chunk_12.npy,490846,6380992,5890146
...,...,...,...,...,...,...,...,...,...
2299,276855,277780,NaN,redreclab__raw_chunk_9_annot_subtones.txt,9,redreclab__raw_chunk_9.npy,490845,4908455,4417610
2300,365576,367078,NaN,redreclab__raw_chunk_9_annot_subtones.txt,9,redreclab__raw_chunk_9.npy,490845,4908455,4417610
2301,429307,430150,NaN,redreclab__raw_chunk_9_annot_subtones.txt,9,redreclab__raw_chunk_9.npy,490845,4908455,4417610
2302,467545,468315,NaN,redreclab__raw_chunk_9_annot_subtones.txt,9,redreclab__raw_chunk_9.npy,490845,4908455,4417610


In [44]:
all["real_start"] = all["cumulative_start"] + all["start"]
all["real_end"] = all["cumulative_start"] + all["end"]
all = all.sort_values("real_start")
all

,start,end,syb,filename,chunk_num,npy_file,size,cumulative_end,cumulative_start,real_start,real_end
2012,256845,258172,NaN,redreclab__raw_chunk_0_annot_subtones.txt,0,redreclab__raw_chunk_0.npy,490845,490845,0,256845,258172
2013,306709,308791,NaN,redreclab__raw_chunk_0_annot_subtones.txt,0,redreclab__raw_chunk_0.npy,490845,490845,0,306709,308791
2014,318915,319687,NaN,redreclab__raw_chunk_0_annot_subtones.txt,0,redreclab__raw_chunk_0.npy,490845,490845,0,318915,319687
2015,320295,321580,NaN,redreclab__raw_chunk_0_annot_subtones.txt,0,redreclab__raw_chunk_0.npy,490845,490845,0,320295,321580
2016,322764,324210,NaN,redreclab__raw_chunk_0_annot_subtones.txt,0,redreclab__raw_chunk_0.npy,490845,490845,0,322764,324210
...,...,...,...,...,...,...,...,...,...,...,...
2148,164925,167282,NaN,redreclab__raw_chunk_66_annot_subtones.txt,66,redreclab__raw_chunk_66.npy,327232,32723038,32395806,32560731,32563088
2149,170650,171520,NaN,redreclab__raw_chunk_66_annot_subtones.txt,66,redreclab__raw_chunk_66.npy,327232,32723038,32395806,32566456,32567326
2150,200313,202146,NaN,redreclab__raw_chunk_66_annot_subtones.txt,66,redreclab__raw_chunk_66.npy,327232,32723038,32395806,32596119,32597952
2151,202913,204166,NaN,redreclab__raw_chunk_66_annot_subtones.txt,66,redreclab__raw_chunk_66.npy,327232,32723038,32395806,32598719,32599972


In [45]:
result = all[["real_start", "real_end", "syb"]]
result

,real_start,real_end,syb
2012,256845,258172,NaN
2013,306709,308791,NaN
2014,318915,319687,NaN
2015,320295,321580,NaN
2016,322764,324210,NaN
...,...,...,...
2148,32560731,32563088,NaN
2149,32566456,32567326,NaN
2150,32596119,32597952,NaN
2151,32598719,32599972,NaN


In [46]:
result.to_csv(base/"labels.txt", header=None, index=None)